In [5]:
import pandas as pd
import numpy as np

In [6]:
###read and viewing dataset
df = pd.read_csv('../data/bank_data.csv')
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            10000 non-null  int64  
 1   x_coordinate  10000 non-null  float64
 2   y_coordinate  10000 non-null  float64
 3   money         10000 non-null  int64  
 4   time (hr)     10000 non-null  float64
dtypes: float64(3), int64(2)
memory usage: 390.8 KB


,id,x_coordinate,y_coordinate,money,time (hr)
0,0,2.851925,1.201712,29700,0.273403
1,1,4.150372,-2.654334,6500,0.064040
2,2,-1.494092,-1.230419,89400,0.127458
3,3,1.271326,-0.088520,96100,1.315029
4,4,2.471113,-0.592810,41100,0.164393
...,...,...,...,...,...
9995,9995,-2.472127,4.787304,2100,0.494904
9996,9996,1.034968,-4.321240,54900,0.492866
9997,9997,-0.512720,-2.949173,32200,0.933033
9998,9998,2.626841,-3.482923,5100,0.551312


In [20]:
###initializing important variables

#24hrs time limit
time_limit = [24, 0, 0]

#travel speed
travel_speed = 30

#escape coordinate point
escape_point = [0, 0]

In [8]:
###distance between 2 points function; where p1 = [x1, y1] and p2 = [x2, y2]. Returns distance in km.
def dist_two_points(p1, p2):
    return np.sqrt(((p2[0] - p1[0])**2) + ((p2[1] - p1[1])**2))

In [9]:
###converting time from hours to hours, minutes and seconds; returns list of integers representing hours, minutes and seconds
 #maybe can return dictionary instead?
def time_convert(time):
    h = int(time // 1)
    m = int((time * 60) - (h * 60))
    s = int((time * 3600) - (h * 3600) - (m * 60))
    
    new_time = [h, m, s]
    
    return new_time

In [10]:
###function to calculate how much time it would take to travel from one bank to another (from p1 to p2). Returns time in 
 #non-converted hours
def calculate_time(p1, p2):
    return time_convert(dist_two_points(p1, p2) / travel_speed)

In [11]:
###sum two time together; for sum of travel time and robbery time
def sum_time(time1, time2):
    time = [0, 0, 0]
    time[0] = time1[0] + time2[0]
    time[1] = time1[1] + time2[1]
    if (time[1] >= 60):
        time[1] = time[1] % 60
        time[0] += 1        
    time[2] = time1[2] + time2[2]
    if (time[2] >= 60):
        time[2] = time[2] % 60
        time[1] += 1
        if (time[1] >= 60):
            time[1] = time[1] % 60
            time[0] += 1 
    return time

In [12]:
###removing time(in hours) from time_limit
def remove_time(time):
    ##when there are hrs, mins and secs to remove
    if (time[0], time[1], time[2] != 0):
        #remove secs from time to time_limit; if negative, remove a minute
        time_limit[2] -= time[2]
        if (time_limit[2] < 0):
            time_limit[1] -= 1
            time_limit[2] += 60
        #remove mins from time to time_limit; if negative, remove an hour    
        time_limit[1] -= time[1]
        if (time_limit[1] < 0):
            time_limit[0] -= 1
            time_limit[1] += 60
        #remove hrs from time to time_limit; if negative, return original time_limit values
        time_limit[0] -= time[0]
        if (time_limit[0] < 0):
            time_limit[0] += time[0]
            time_limit[2] += time[2]
            if (time_limit[2] >= 60):
                time_limit[2] = time_limit[2] % 60
                time_limit[1] += 1  
            time_limit[1] += time[1]
            if (time_limit[1] >= 60):
                time_limit[1] = time_limit[1] % 60
                time_limit[0] += 1
                
    ##when there are only hours and mins to remove
    elif ((time[2] == 0) and (time[0], time[1] != 0)):
        #remove mins from time to time_limit
        time_limit[1] -= time[1]
        if (time_limit[1] < 0):
            time_limit[0] -= 1
            time_limit[1] += 60
        #removes hours from time to time_limit; if negative, return original time_limit
        time_limit[0] -= time[0]
        if (time_limit[0] < 0):
            time_limit[0] += time[0]
            time_limit[1] += time[1]
            if (time_limit[1] >= 60):
                time_limit[1] = time_limit[1] % 60
                time_limit[0] += 1

    ##when there are only hours and secs to remove
    elif ((time[1] == 0) and (time[0], time[2] != 0)):
        #remove secs from time to time_limit; if negative, remove a minute
        time_limit[2] -= time[2]
        if (time_limit[2] < 0):
            time_limit[1] -= 1
            time_limit[2] += 60
        #if mins is negative, remove an hour and add 60 minutes
        if (time_limit[1] < 0):
            time_limit[0] -= 1
            time_limit[1] += 60
        #remove hrs from time to time_limit; if negative, return original time_limit values
        time_limit[0] -= time[0]
        if (time_limit[0] < 0):
            time_limit[0] += time[0]
        time_limit[2] += time[2]
        if (time_limit[2] >= 60):
            time_limit[2] = time_limit[2] % 60
            time_limit[1] += 1

    ##when there are only mins and secs to remove
    elif ((time[0] == 0) and (time[1], time[2] != 0)):
        #remove secs from time to time_limit; if negative, remove a minute        
        time_limit[2] -= time[2]
        if (time_limit[2] < 0):
            time_limit[1] -= 1
            time_limit[2] += 60
        #remove mins from time to time_limit; if negative, return the original time_limit values if there are no hours left in 
         #time_limit, if there are still hours left in time_limit, remove an hour
        time_limit[1] -= time[1]
        if ((time_limit[1] < 0) and (time_limit[0] > 0)):
            time_limit[1] += 60 
            time_limit[0] -= 1
        elif ((time_limit[1] < 0) and (time_limit[0] == 0)):
            time_limit[1] += time[1]
            time_limit[2] += time[2]
            if (time_limit[2] >= 60):
                time_limit[2] = time_limit[2] % 60
                time_limit[1] += 1
            
    ##when there are only hrs to remove
    elif (time[0] != 0 and time[1],time[2] == 0):
        time_limit[0] -= time[0]
        if (time_limit[0] < 0):
            time_limit[0] += time[0]
    
    ##when there are only mins to remove
    elif (time[1] != 0 and time[0],time[2] == 0):
        time_limit[1] -= time[1]
        if ((time_limit[1] < 0) and (time_limit[0] > 0)):
            time_limit[0] -= 1
            time_limit[1] += 60
        elif ((time_limit[1] < 0) and (time_limit[0] == 0)):
            time_limit[1] += time[1] 
    
    ##when there are only secs to remove
    elif (time[0], time[1] == 0 and time[2] != 0):
        time_limit[2] -= time[2]
        if ((time_limit[2] < 0) and (time_limit[1] > 0) and (time_limit[0] == 0)):
            time_limit[1] -= 1
            time_limit[2] += 60
        elif ((time_limit[2] < 0) and (time_limit[0], time_limit[1] > 0)):
            time_limit[1] -= 1
            time_limit[2] += 60
        elif ((time_limit[2] < 0) and (time_limit[0] > 0) and (time_limit[1] == 0)):
            time_limit[0] -= 1
            time_limit[1] = 59
            time_limit[2] += 60
        elif ((time_limit[2] < 0) and (time_limit[0], time_limit[1] == 0)):
            time_limit[2] += time[2]           

    return time_limit

In [13]:
#make coordinate points using x & y coordinates values in df; easier when using dist_two_points and calculate_time 
df1 = df.copy()
df1['coordinate_point'] = df1[['x_coordinate','y_coordinate']].values.tolist()
df1

,id,x_coordinate,y_coordinate,money,time (hr),coordinate_point
0,0,2.851925,1.201712,29700,0.273403,"[2.851925250594566, 1.2017116385810744]"
1,1,4.150372,-2.654334,6500,0.064040,"[4.150371890795688, -2.6543343395758168]"
2,2,-1.494092,-1.230419,89400,0.127458,"[-1.494092041666938, -1.2304190057197617]"
3,3,1.271326,-0.088520,96100,1.315029,"[1.2713257776543574, -0.0885204586243504]"
4,4,2.471113,-0.592810,41100,0.164393,"[2.4711126193849884, -0.5928099315295077]"
...,...,...,...,...,...,...
9995,9995,-2.472127,4.787304,2100,0.494904,"[-2.472126859357382, 4.787303515861167]"
9996,9996,1.034968,-4.321240,54900,0.492866,"[1.034968293560974, -4.321240418799071]"
9997,9997,-0.512720,-2.949173,32200,0.933033,"[-0.5127201881002563, -2.9491731088900157]"
9998,9998,2.626841,-3.482923,5100,0.551312,"[2.626840581796636, -3.482922651575543]"


In [14]:
#make money/time (hr) ratio (m/t_ratio)column for greedy algorithm
df1['m/t_ratio'] = df1['money'] / df1['time (hr)']
df1

,id,x_coordinate,y_coordinate,money,time (hr),coordinate_point,m/t_ratio
0,0,2.851925,1.201712,29700,0.273403,"[2.851925250594566, 1.2017116385810744]",108630.921470
1,1,4.150372,-2.654334,6500,0.064040,"[4.150371890795688, -2.6543343395758168]",101499.543603
2,2,-1.494092,-1.230419,89400,0.127458,"[-1.494092041666938, -1.2304190057197617]",701405.770444
3,3,1.271326,-0.088520,96100,1.315029,"[1.2713257776543574, -0.0885204586243504]",73078.228687
4,4,2.471113,-0.592810,41100,0.164393,"[2.4711126193849884, -0.5928099315295077]",250010.909373
...,...,...,...,...,...,...,...
9995,9995,-2.472127,4.787304,2100,0.494904,"[-2.472126859357382, 4.787303515861167]",4243.243258
9996,9996,1.034968,-4.321240,54900,0.492866,"[1.034968293560974, -4.321240418799071]",111389.317396
9997,9997,-0.512720,-2.949173,32200,0.933033,"[-0.5127201881002563, -2.9491731088900157]",34511.098257
9998,9998,2.626841,-3.482923,5100,0.551312,"[2.626840581796636, -3.482922651575543]",9250.652010


In [15]:
#sorting sample by m/t_ratio
sorted_df1 = df1.sort_values(by = 'm/t_ratio', ascending = False)
sorted_df1

,id,x_coordinate,y_coordinate,money,time (hr),coordinate_point,m/t_ratio
3613,3613,-1.950527,-1.495858,54400,0.000186,"[-1.95052692211966, -1.4958578401494482]",2.929603e+08
9546,9546,-1.810721,-0.795421,75100,0.000788,"[-1.8107213688092736, -0.7954212248596171]",9.530556e+07
3803,3803,3.867586,0.918332,18500,0.000358,"[3.8675857182610422, 0.918331555282218]",5.169381e+07
6528,6528,-0.282937,3.224334,33900,0.000769,"[-0.2829367058853993, 3.2243343253980616]",4.407762e+07
9583,9583,3.393835,-2.790830,24000,0.001131,"[3.393835414529165, -2.790830217677157]",2.121396e+07
...,...,...,...,...,...,...,...
1838,1838,-2.179862,-3.064824,100,1.486872,"[-2.179861681402513, -3.0648236481105826]",6.725528e+01
8832,8832,1.934915,-4.278272,100,1.490015,"[1.934914768174492, -4.278271951584535]",6.711343e+01
4764,4764,0.565196,1.437413,100,1.491462,"[0.5651957860547507, 1.4374129496693444]",6.704829e+01
2123,2123,-3.105083,-1.135305,100,1.494467,"[-3.1050830448776914, -1.1353049322398934]",6.691347e+01


In [21]:
#approach is going backwards; from escape point to first bank
#calculate how much time it takes from last bank to escape point before going through the greedy algorithm
first_travel_time_to_remove = calculate_time(sorted_df1['coordinate_point'].iloc[0], [0,0])
#calculate time needed to rob the last bank 
first_rob_time = time_convert(sorted_df1['time (hr)'].iloc[0])
#add both times using sum_time
first_time_to_remove = sum_time(first_travel_time_to_remove, first_rob_time)
#removing first_time_to_remove from time_limit
remove_time(first_time_to_remove)

[23, 55, 6]

In [22]:
#greedy algorithm

#initializing end point to coordinate point of first element of sorted_df1
end_point = sorted_df1['coordinate_point'].iloc[0]

#intializing variable to store banks' id; starting with the first element of sorted_df1
banks_id = [sorted_df1['id'].iloc[0]]

#for every element starting from the second element, calculate time to travel from start_point to end_point and time to rob bank
j = 1
while j < 10000:
    #creating variable time_left to compare later with time_limit
    time_left = str(time_limit)
    
    #marking the start point for time travel calculation
    start_point = sorted_df1['coordinate_point'].iloc[j]
    
    #calculate sum of travel time from start to end points and robbery time
    travel_time = calculate_time(start_point, end_point)
    rob_time = time_convert(sorted_df1['time (hr)'].iloc[j])
    total_time = sum_time(travel_time, rob_time)
    
    #remove time from time_limit; if removed time is larger than time_limit, it automatically returns the initial time_limit
    remove_time(total_time)
    
    #verify that there is still enough time by verifying that the time before is not the same as before (time_limit != time_left)
    if (time_left != str(time_limit)):  
        banks_id.append(sorted_df1['id'].iloc[j]) #store bank id in banks_id
        end_point = sorted_df1['coordinate_point'].iloc[j] #make new end_point as the 'coordinate_point' of j
        j += 1
    #if time_limit == time_left, continue and try with next bank
    elif (time_left == str(time_limit)):
        j += 1
        
#print list with banks' id        
banks_id       

[3613,
 9546,
 3803,
 6528,
 9583,
 5933,
 9195,
 4757,
 8550,
 4499,
 9241,
 7343,
 3798,
 7544,
 4762,
 8436,
 8562,
 6097,
 487,
 2928,
 5135,
 865,
 3914,
 4987,
 3466,
 7560,
 8579,
 8690,
 8469,
 433,
 524,
 6468,
 1914,
 1757,
 8287,
 7258,
 4725,
 8206,
 4605,
 7064,
 3926,
 9401,
 9736,
 2331,
 9653,
 7074,
 2346,
 8966,
 4696,
 8022,
 4789,
 6740,
 9881,
 70,
 2656,
 951,
 209,
 58,
 9228,
 3193,
 8525,
 5610,
 2037,
 4287,
 6022,
 2827,
 8849,
 9378,
 6104,
 2741,
 7701,
 8355,
 5563,
 3005,
 7665,
 6987,
 5719,
 2729,
 1997,
 8286,
 4906,
 613,
 790,
 7087,
 5126,
 5562,
 3516,
 2243,
 6759,
 3026,
 1372,
 2442,
 1733,
 8375,
 7764,
 7649,
 7689,
 9880,
 3297,
 4794,
 3089,
 7583,
 5725,
 517,
 4345,
 2521,
 2190,
 5155,
 9049,
 444,
 6712,
 8908,
 8703,
 7877,
 7595,
 8231,
 9275,
 4465,
 5295,
 5296,
 299,
 8846,
 6254,
 507,
 6535,
 6216,
 279,
 5166,
 6764,
 3582]

In [23]:
#check time left
time_limit

[0, 0, 35]

In [24]:
#check how much money was acquired
total_money = 0
for i in banks_id:
    total_money += sorted_df1['money'].loc[i]
total_money

6451100